In [ ]:
# to evaluate the error
# negative error never happens
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter
from datetime import datetime as dt
import statistics
from numpy import linspace

DATASETS_DIR = "../tests/input_files/"
# different sets for precision and performance tests
DATASETS =['test_100_25_15', 'test_100_25_25', 'test_100_25_35', 'test_100_25_50',
           'test_100_25_75', 'test_100_50_15', 'test_100_50_25', 'test_100_50_35',
           'test_100_50_50', 'test_100_50_75']
# DATASETS = os.listdir(DATASETS_DIR)
print(DATASETS)

In [ ]:
# run and collect data
CMD_TEMPL = "../CSP {} {} -v 1"
dataset_data = {}
t0 = dt.now()

def get_answer(ans, filename, f=None, z=None):
    """Get the smallest program answer."""
    cmd = CMD_TEMPL.format(filename, ans)
    if z:
        cmd += " -z"
    if f:
        cmd += " -f"
    csp_out = subprocess.check_output(cmd, shell=True).decode("utf-8").split("\n")
    if "True" in csp_out:
        # ok, solved
        return ans
    max_cov = 0
    req_cov = 0
    for line in csp_out:
        line_data = line.split()
        if line.startswith("Maximal coverage found is"):
            max_cov = int(line_data[-3])
            req_cov = int(line_data[-1])
            return ans + req_cov - max_cov
        if line.startswith("# Cannot find initial move"):
            return 100
            

datasets_num = len(DATASETS)
for num, dataset in enumerate(DATASETS):
    t_d0 = dt.now()
    dataset_metadata = dataset.split("_")
    str_len = int(dataset_metadata[1])
    str_num = int(dataset_metadata[2])
    answer = int(dataset_metadata[3])
    dataset_data[num] = {"answers_0": [],
                         "answers_f": [],
                         "answers_z": [],
                         "answers_zf": [],
                         "data": {"str_len": str_len,
                                  "str_num": str_num,
                                  "answer": answer}}
    dataset_dir = os.path.join(DATASETS_DIR, dataset)
    contents = os.listdir(dataset_dir)
    for fnum, fle in enumerate(contents):
        f_path = os.path.join(dataset_dir, fle)

        k_0 = get_answer(answer, f_path)
        k_f = get_answer(answer, f_path, f=True)
        k_z = get_answer(answer, f_path, z=True)
        k_zf = get_answer(answer, f_path, z=True, f=True)
        if not k_0 or not k_f or not k_z or not k_zf:
            print(f_path)

        dataset_data[num]["answers_0"].append(k_0)
        dataset_data[num]["answers_f"].append(k_f)
        dataset_data[num]["answers_z"].append(k_z)
        dataset_data[num]["answers_zf"].append(k_zf)
    print(f"Dataset {num + 1} / {datasets_num} done in {dt.now() - t_d0}")

print(f"Time spent: {dt.now() - t0}")

In [ ]:
# compare answers with and without F flag
all_answers = []
all_f_answers = []
all_b_answers = []

fig = plt.figure(figsize=(24, 7))
ax_1 = fig.add_subplot(1, 3, 1)
ax_2 = fig.add_subplot(1, 3, 2)
ax_3 = fig.add_subplot(1, 3, 3)

for k, v in dataset_data.items():
    ax_1.scatter(v["answers_0"], v["answers_f"], s=15)
    ax_2.scatter(v["answers_0"], v["answers_z"], s=15)
    ax_3.scatter(v["answers_0"], v["answers_zf"], s=15)

x = linspace(*ax_1.get_xlim())
ax_1.plot(x, x, "--", color="black", alpha=0.5)
ax_1.set_xlabel("Answer without correction")
ax_1.set_ylabel("Answer with correction F")
ax_1.grid()

x = linspace(*ax_2.get_xlim())
ax_2.plot(x, x, "--", color="black", alpha=0.5)
ax_2.set_xlabel("Answer without correction")
ax_2.set_ylabel("Answer with correction Z")
ax_2.grid()

x = linspace(*ax_3.get_xlim())
ax_3.plot(x, x, "--", color="black", alpha=0.5)
ax_3.set_xlabel("Answer without correction")
ax_3.set_ylabel("Answer with corrections Z and F")
ax_3.grid()

plt.show()

In [ ]:
# visualize the error
for k, v in dataset_data.items():
    v["errors_zf"] = [a - v["data"]["answer"] for a in v["answers_zf"]]
    v["errors_z"] = [a - v["data"]["answer"] for a in v["answers_z"]]
    v["errors_f"] = [a - v["data"]["answer"] for a in v["answers_f"]]
    v["errors_0"] = [a - v["data"]["answer"] for a in v["answers_0"]]

ERR_LIM = 25  # ignore errors > 25% for better plotting
fig = plt.figure(figsize=(20, 10))
to_plot = []
titles = []

perc_errors_0 = []
perc_errors_f = []
perc_errors_z = []
perc_errors_zf = []
answers = []

for num, (k, v) in enumerate(dataset_data.items(), 1):
    # ax = fig.add_subplot(rows, cols, num)
    ans = v["data"]["answer"]
    ans_arr = [ans for _ in range(len(v["errors_0"]))]
    answers.extend(ans_arr)

    perc_error_0 = [e / ans * 100 if e / ans * 100 < ERR_LIM else ERR_LIM for e in v["errors_0"]]
    perc_error_f = [e / ans * 100 if e / ans * 100 < ERR_LIM else ERR_LIM for e in v["errors_f"]]
    perc_error_z = [e / ans * 100 if e / ans * 100 < ERR_LIM else ERR_LIM for e in v["errors_z"]]
    perc_error_zf = [e / ans * 100 if e / ans * 100 < ERR_LIM else ERR_LIM for e in v["errors_zf"]]

    perc_errors_0.extend(perc_error_0)
    perc_errors_f.extend(perc_error_f)
    perc_errors_z.extend(perc_error_z)
    perc_errors_zf.extend(perc_error_zf)

    to_plot.append(perc_error_0)
    to_plot.append(perc_error_f)
    to_plot.append(perc_error_z)
    to_plot.append(perc_error_zf)

    title = "{} {} {}".format(v["data"]["str_len"], v["data"]["str_num"], ans)
    titles.append(title)
    titles.append(title + " F")
    titles.append(title + "Z")
    titles.append(title + "ZF")

ax_1 = fig.add_subplot(1, 1, 1)
pos = list(range(1, len(titles) + 1))
vp = ax_1.violinplot(to_plot, pos,
                     showmeans=True,
                     showextrema=True,
                     showmedians=True)

ax_1.set_title("Errors with and without a correction")
ax_1.set_ylabel("Error %")
ax_1.axhline(y=25, color='r', linestyle='-')
ax_1.set_xticks(pos)
ax_1.set_xticklabels(titles, rotation=45, fontsize=8)
ax_1.grid(color='grey', linestyle='--', linewidth=0.5)

colors = ["navy", "navy", "navy", "navy",
          "purple", "purple", "purple", "purple",
          "orange", "orange", "orange", "orange",
          "green", "green", "green", "green",
          "pink", "pink", "pink", "pink",
          ] * 2
vp["cmeans"].set_edgecolor("blue")
vp["cmedians"].set_edgecolor("green")
vp["cbars"].set_edgecolor("grey")
vp["cmaxes"].set_edgecolor("grey")
vp["cmins"].set_edgecolor("grey")

for vb, color in zip(vp['bodies'], colors):
    vb.set_facecolor(color)
    vb.set_edgecolor("grey")
plt.show()

print("Mean uncorr error: ", statistics.mean(perc_errors_0))
print("Median uncorr error: ", statistics.median(perc_errors_0))
print("Mean corr F error: ", statistics.mean(perc_errors_f))
print("Median corr F error: ", statistics.median(perc_errors_f))
print("Mean corr Z error: ", statistics.mean(perc_errors_z))
print("Median corr Z error: ", statistics.median(perc_errors_z))
print("Mean corr ZF error: ", statistics.mean(perc_errors_zf))
print("Median corr ZF error: ", statistics.median(perc_errors_zf))
# errors > 25% are to be TOO MUCH

In [ ]:
no_err = perc_errors_f.count(0)
print(f"No errors: {no_err} measurements")
print(f"Errors in: {len(perc_errors_f) - no_err} measurements")